<h1> Datto RMM - CSV import - Filter by deviceID - Shape - Export to PostgreSQL </h1>


# Import Modules and Define Functions
## Import Modules
### Import DataFrame and Shaping Modules

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import json
import re

# import configparser for env secrets
from configparser import ConfigParser

config = ConfigParser()
config.read('d:/git/example_infrastructure_data_dev/config/env.ini')
import requests
from requests.structures import CaseInsensitiveDict

### Import PostgreSQL Modules

In [ ]:
# import necessary packages
import psycopg2 as pg
from sqlalchemy import create_engine

In [ ]:
# import and assign secrets from env.ini

# postgreSQL
username = config['postgresql']['username']
password = config['postgresql']['password']
uri = config['postgresql']['uri']
port = config['postgresql']['port']

In [ ]:
# define if you are pulling raw data ('data_pool') or test data ('seed_data')
database = 'seed_data'

# exclude csv if missing column name
exclude_on_missing_column = False

# DEFINE THE DB URI
db_uri = f'postgresql://{username}:{password}@{uri}:{port}/{database}'

# DEFINE THE ENGINE (CONNECTION OBJECT)
engine = create_engine(db_uri, echo=True)


## Define Functions

In [ ]:
import os
import datetime

def add_time(source_file):
    source_mfdate = 'Source Modified Date'
    source_crdate = 'Source Creation Date'
    source_fn = 'Source Filename'

    # Both the variables would contain time
    # elapsed since EPOCH in float
    ti_c = os.path.getctime(source_file)
    ti_m = os.path.getmtime(source_file)
    fi_n = os.path.basename(source_file)

    # Converting the time in seconds to UTC datetime
    c_ti = datetime.datetime.utcfromtimestamp(ti_c).strftime('%Y/%m/%d %H:%M:%S')
    m_ti = datetime.datetime.utcfromtimestamp(ti_m).strftime('%Y/%m/%d %H:%M:%S')


    return {source_crdate:c_ti,source_mfdate:m_ti,source_fn:fi_n}

## Arguments and Declarations


### Define Source and Export Folders

In [ ]:
# identify folder stages so that files are not called twice in the same stage
source_dir = 'D:/users/gmcwilliams/downloads/'
# source_dir = 'D:\cloud_storage\OneDrive - Think Stack\Reports Automation'

# export folder will contain all csv exported DataFrames
export_folder = 'd:/exports/'

### Define Current Time of Script Execution

In [ ]:
# add current timestamp to filename for reference
current_time = (dt.datetime.utcnow().strftime('%Y_%m_%d_%H%M%S'))

### Initialize and Define Structure and Containers for Data Shaping

In [ ]:
# define key column to join on
fieldnames_to_compare = 'Device UID'

In [ ]:
# dictionary of df_names and queries used to create tables in postreSQL
device_details = {'dataframe':'device_details','query':'SELECT * FROM datto_rmm.device_details'}
os_patch_mgmt = {'dataframe':'os_patch_mgmt','query':'SELECT * FROM datto_rmm.os_patch_mgmt'}

import_dataframes = [device_details,os_patch_mgmt]

In [ ]:

# Selected Columns from Master Device View Export CSV - This is the column mask that will be used to trimp the outer merge on match key column
std_columns = [
    'Device UID',
    'Site Name',
    'Site UID',
    'Device Hostname',
    'Create Date',
    'Last Seen',
    'Last Audit Date',
    'Policy',
    'Patches Approved Pending',
    'Patches Not Approved',
    'Patches Installed',
    'Patch Status',
    'Schedule',
    'Last Run',
    'Operating System',
    'Device CPU',
    'Physical CPU Cores',
    '.NET Version',
    'Memory',
    'Device Type',
    'Domain',
    'Disk Drive (total/free)',
    'Online Duration (hrs)',
    'Architecture',
    'Last Reboot',
    'Reboot required',
    'Int IP Address',
    'User-Defined Field 10',
    'MAC Address(es)',
    'Software Status',
    'Group',
    'Antivirus Product',
    'Antivirus Status',
    'Source Modified Date',
    'Source Creation Date',
    'Source Filename'
]

device = [
    'Device UID',
    'Device Hostname',
    'Site UID',
    'Site Name',
    'Device Description',
    'Int IP Address',
    'Ext IP Addr',
    'Create Date',
    'Last Seen',
    'Last Audit Date',
    'Session Name',
    'Privacy Mode',
    'Agent Version',
    'Device Model',
    'Operating System',
    'Serial Number',
    'Motherboard',
    'Device CPU',
    'Physical CPU Cores',
    '.NET Version',
    'Memory',
    'MAC Address(es)',
    'User-Defined Field 10',
    'Device Type',
    'Domain',
    'Disk Drive (total/free)',
    'Online Duration (hrs)',
    'Architecture',
    'BIOS Name',
    'BIOS Release Date',
    'BIOS Version',
    'Last Reboot',
    'Reboot required',
    'Manufacturer',
]

manage = [
    'Device UID',
    'Site Name',
    'Device Hostname',
    'Device Description',
    'Policy',
    'Int IP Address',
    'Ext IP Addr',
    'Last User',
    'Group',
    'Create Date',
    'Last Seen',
    'Last Audit Date',
    'Session Name',
    'Agent Version',
    'Operating System',
    'Service Pack',
    'Serial Number',
    'User-Defined Field 10',
    'Last Run',
    'Schedule',
    'Patch Status',
    'Patches Approved Pending',
    'Patches Installed',
    'Patches Not Approved',
    'Device Type',
    'Domain',
    'Disk Drive (total/free)',
    'Online Duration (hrs)',
    'Last Reboot',
    'Reboot required',
    'Manufacturer'
    ]

In [ ]:
# output of csv with matching key column
included_files = {}

# output csv of all files that could not be merged
excluded_files = {}

# Regex Match to group files to be combined on rows rather than merged on columns to prevent dropping rows if there isnt a key column match when files are combined in random order
pattern = re.compile(r'^(\w+)_')

# CSV File Types
devices_tab_export_filename = 'DeviceDetailsExport'
manage_tab_export_filename = 'SystemDeviceSelection'
grouped_export_files_list = []
shaped_df_object_list = []

# Read all files in source_dir and sub directories
## Filter by '.csv'

In [ ]:
# pull all filenames walking through all folders (recursive going down the tree)
source_csv_dict = {}
for root, dirs, files in os.walk(source_dir):
    for file in files:
        if '.csv' in file:
            source_csv_dict.update({os.path.join(file):os.path.join(root,file)})
            

In [ ]:
# print(all_source_csv)
print('All CSV Files found before futher vetting and filtering')
print('='*50)
for k,v in source_csv_dict.items():
    print(k)

# Sorting and Excluding Files

## Read all csv file columns and create two lists of files:
### Those with the chosen merge key column will be kept and the remaining filenames will not be called any further

In [ ]:
for k,v in source_csv_dict.items():
    if exclude_on_missing_column == True:  
        df = pd.read_csv(v)
        if fieldnames_to_compare not in df.columns:
            print(f'Missing Key: {fieldnames_to_compare} to Join in {k}')
            excluded_files.update({k:v})
        else:
            included_files.update({k:v})
    else:
        included_files.update({k:v})

In [ ]:
print('Files with CORRECT join key column:')
print('-'*50)
for file in included_files:
    print(file)
print('='*50)

print('Files MISSING join key column:')
print('-'*50)
for file in excluded_files:
    print(file)
print('='*50)

## Parse Accepted CSV's for file discription and store as dictionary key pair

In [ ]:
pattern = re.compile(r'^([a-zA-Z]{0,})(\_|\-|''){0,1}([a-zA-Z]{0,})')

In [ ]:
for k,v in included_files.items():
    print(k)

In [ ]:
for k,v in included_files.items():
    matches = pattern.search(k)
    if matches[1] == devices_tab_export_filename:
        #print(f'''['{v}'] matches: ['{devices_tab_export_filename}'] on ['{matches[1]}']''')
        grouped_export_files_list.append({'filename':v,'groupname':'device_details','columns':device})
    elif matches[3] == manage_tab_export_filename:
        #print(f'''['{v}'] matches: ['{manage_tab_export_filename}'] on ['{matches[3]}']''')
        grouped_export_files_list.append({'filename':v,'groupname':'os_patch_mgmt','columns':manage})

    
print('='*50)

In [ ]:
for k in grouped_export_files_list:
    print(k)

## For those files that have the key column, shape add source info

### 1. Add in any missing columns against the standard so data columns line up on import
### 3. Trim extra df columns to match column standards
### 4. Replace any known type mismatch values before setting datetime
### 5. Add source file data as columns at end of dataframe (record the file creation, modified, and fullpath name)
### 6. Parse known date columns to datetime so the types are correct in db import
### 7. EXPORT to postregsql

In [ ]:
for file in grouped_export_files_list:
    filename = file['filename']
    groupname = file['groupname']
    grp_columns = file['columns']


    # pull source time from file properties
    source_info = add_time(filename)


    # NA Values Check
    na_values = ['Currently Online','null', '(null)']


    # Import CSV to Pandas
    print(f"reading file ['{filename}'] and ['{groupname}']!")
    #,index_col=['Device UID','Source Modified Date']
    df = pd.read_csv(filename,na_values=na_values,skipinitialspace=True)


    # if column is missing in dataframe add it before upload to prevent mismatch or multi indexed columns
    for c in grp_columns:
        if c not in df:
            df[c] = np.nan

    # DTYPES

    # prepare DTYPE values to match predicted values - get column initial dtypes
    column_dtypes = dict(df.dtypes)

    for k,v in column_dtypes.items():

        # strip any whitespace from object columns (non-datetime or boolean)
        if v == 'object':
            df[k].str.strip()

        # Condition boolean values for postgreSQL: (True,False,NULL) Only!
        elif v == 'bool':
            df[k].mask(df[k] == '', pd.NA, inplace=True)

    # regex remove whitespace
    df = df.replace(r'^\s+$', np.nan, regex=True)


    # Replacement Values for type mismatch
    replace_dict = {
        'Currently Online':source_info['Source Modified Date'],
        '':pd.NA
    }
    df.replace(replace_dict)


    # drop non-standard columns
    df.drop([col for col in df.columns if col not in grp_columns], axis=1, inplace=True)
    #print(f'columns after drop: {df.columns}')




    # add source info to new columns k with values v
    for k, v in source_info.items():
        df[k] = v


    # Parse Date Data Options
    date_parser = lambda c: pd.to_datetime(c, errors='coerce')
    parse_dates =  [
        'Create Date',
        'Last Seen',
        'Last Reboot',
        'Source Creation Date',
        'Source Modified Date'
    ]

    # filename prefix timestamp format
    time_format = '%Y_%m_%d_%H%M%S'


    # convert dates to datetime
    df[parse_dates] = df[parse_dates].apply(date_parser)


    # section can be uncommented for seed data creation for import on a new table to set column names and types
    #df.to_csv(export_folder + 'csv_ingress_' + str(current_time) + '.csv')
    df.to_json(export_folder + 'csv_ingress_' + str(current_time) + '.json')


    # export to postgresql
    #df.to_sql(groupname, con=engine, if_exists='replace',  schema='datto_rmm')
    print('='*100)
    print('')
    print('='*100)

In [ ]:
df